In [1]:
import pandas as pd
from utils import config

from sklearn.model_selection import train_test_split

total_df = pd.read_excel(config.xls_path)
total_df = total_df.sort_values(by=['lemm_review_len','overlap'], ascending = False)
train_df, val_df = train_test_split(total_df, test_size=0.1, 
                                    random_state=0, shuffle=True)

In [2]:
def longest_common_subsequence(main_string, comparing_string):

#     main_string = main_string.split(" ")
#     comparing_string = comparing_string.split(" ")
    columns_length = len(main_string)  # Get the length of the first word or base word
    rows_length = len(comparing_string)  # Get the length of the second word or comparing word

    # MAKE A 2D LIST (MATRIX)
    dynamic_table = [[0] * (columns_length + 1) for i in range(rows_length + 1)]

    # rows_length = NUMBER OF ROWS
    # columns_length = NUMBER OF COLUMNS
    
    # FILL THE MATRIX FOLLOWING LCS ALGORITHM.
    for i in range(1, rows_length + 1):
        for j in range(1, columns_length + 1):
            if main_string[j - 1] == comparing_string[i - 1]:
                dynamic_table[i][j] = 1 + dynamic_table[i - 1][j - 1]

            else:
                dynamic_table[i][j] = max(dynamic_table[i - 1][j], dynamic_table[i][j - 1])

#     print("MATRIX ACCORDING TO LONGEST COMMON SUBSEQUENCE ALGORITHM: \n ")

    # for i in range(rows_length + 1):
    #     print(dynamic_table[i])

    #print("LENGTH OF LONGEST COMMON SUBSEQUENCE = ", dynamic_table[rows_length][columns_length])

    len_lcs = dynamic_table[rows_length][columns_length]

    i = len(comparing_string)
    j = len(main_string)

    lcs_string = str()

    # BACKTRACKING TO FIND THE LONGEST COMMON SUBSEQUENCE

    temp = True

    while temp is True:
        if dynamic_table[i][j] == 0:
            temp = False
        elif dynamic_table[i][j] == dynamic_table[i][j - 1]:
            j = j - 1

        else:
            lcs_string = main_string[j-1] + " " + lcs_string
            i = i - 1
            j = j - 1

    return lcs_string, len_lcs

In [3]:
# df.loc[i,'token_lcs'] = token_lcs
#             df.loc[i,'len_lcs'] = len_lcs
#             df.loc[i,'percent_lcs'] = percent_lcs
#             df.loc[i,'overlap_pos'] = overlap_pos
            
#             df.loc[i,'reveiw_len'] = len(rev_tokens)
#             df.loc[i,'summary_len'] = len(summary_tokens)

In [4]:
#%%
from tqdm import tqdm
from copy import deepcopy
def analay(df, mode):
    df['token_lcs'] = 0
    df['len_lcs'] = 0
    df['percent_lcs'] = 0
    df['overlap_pos'] = 0

    df['reveiw_len'] = 0
    df['summary_len'] = 0
#     df = df[:100]
    with tqdm(total=len(df)) as pbar:
        for i ,row in df.iterrows():       
            overlap_pos = None

            rev, summary = row['review'], row['summary']
            rev_tokens, summary_tokens = rev.split(" "), summary.split(" ")           
            token_lcs, len_lcs = longest_common_subsequence(rev_tokens, summary_tokens)
            percent_lcs = len_lcs / len(summary_tokens) * 100 # percent

            rev_sents = rev.split(" . ")
            overlap_sents = {idx: longest_common_subsequence(sent.split(" "), summary_tokens) for idx, sent in enumerate(rev_sents)}
            sort_overlap_sent = {k:v for k,v in sorted(overlap_sents.items(), key= lambda item: item[1], reverse=True)}
            overlap_sent_id = list(sort_overlap_sent.items())[0][0]

            if (overlap_sent_id == 0) and (percent_lcs >= 20): overlap_pos = 0 # overlap_Top 
            elif (overlap_sent_id == len(rev_tokens)-1) and (percent_lcs >= 20): overlap_pos = 1 # overlap_Final 
            elif (percent_lcs >= 20): overlap_pos = 2 # overlap_other 
            else: overlap_pos = -1 # no overlap 

            df.loc[i,'token_lcs'] = token_lcs
            df.loc[i,'len_lcs'] = len_lcs
            df.loc[i,'percent_lcs'] = percent_lcs
            df.loc[i,'overlap_pos'] = overlap_pos
            
            df.loc[i,'reveiw_len'] = len(rev_tokens)
            df.loc[i,'summary_len'] = len(summary_tokens)

            pbar.update(1)
            pbar.set_description("%s row: %s | overlap_pos: %s (%2.2f)percent| len_lcs: %s" % (mode, i, overlap_pos, percent_lcs, len_lcs))
#     df = df[df['token_lcs']!=0]
    return df



In [5]:
train_df.head()

,review,summary,cheat,cheat_num,overlap,lemm_review_len,lemm_summary_len,POS_FOP_keywords,DEP_FOP_keywords,TextRank_keywords
84365,the quality great and very good size . the onl...,<s> great for the price wish you could have go...,"{'gold', 'simultaneously', 'side', 'silver', '...",5,9,175,16,"size very good,size good,side opposite,cloth o...","size great,side opposite,cloth entire,cloth ou...",thing like switch definitely recommend work si...
203843,this great product for home rehab . put desk f...,<s> this is great product for at home rehab </s>,"{'rehab', 'product', 'great'}",3,6,49,10,product great,"product great,time tension",great product home
429790,purchase this product after 5610 stop work . m...,<s> poor scanning construction no duplex suppo...,"{'support', 'scanning', 'duplex', 'construction'}",4,4,154,10,"product purchase,unit return,printing sided,si...","side opposite,paper standard,barrier horizonta...",make scanning scan sheet barrier number motor ...
349897,the phone receive was the same make and model ...,<s> the cell receive does not match what order...,"{'cell', 'receive', 'order'}",3,6,92,10,"titanium pink,cell sell","blackberry unlocked,titanium pink,request mode...",pink phone make titanium order
54691,setup easy enough altho the camera mount rathe...,<s> setup is easy enough altho the camera moun...,"{'setup', 'flimsy', 'enough', 'easy', 'camera'...",6,9,40,13,"picture daytime,image nighttime",picture daytime,setup easy camera


In [6]:
val_df.head()

,review,summary,cheat,cheat_num,overlap,lemm_review_len,lemm_summary_len,POS_FOP_keywords,DEP_FOP_keywords,TextRank_keywords
138833,latest update review wallpixel app has now dec...,<s> important review note good camera easy set...,"{'good', 'setup', 'easy', 'camera', 'review', ...",6,8,235,15,"review initial,setup very easy,setup easy,rang...","camera app,year half,review initial,hardware e...",like review app camera setup hardware home eng...
461231,update 2017 request replacement gasket for the...,<s> lid gasket fall out after two month use ot...,"{'container', 'lid', 'gasket', 'cat', 'fall', ...",6,10,192,16,"wipe easy,raccoon paw,hold full,bag full,cat d...","wipe easy,mark paw,bag full,food dry,maneuver ...",lid replacement gasket replace easy raccoon gr...
443223,this very simple setup . after unbox remove al...,<s> print quality is good it print quickly </s>,"{'good', 'quality', 'print'}",3,5,190,9,"setup very simple,setup simple,bit protective,...","setup simple,bit protective,iphone ipad,print ...",wizard screen simple setup protective bit unbo...
347768,these are the worst tasting thing have ever ta...,<s> these are the worst tasting thing have eve...,"{'worst', 'thing', 'tasting', 'taste'}",4,9,23,11,"thing tasting,idea taste",thing tasting,taste tasting thing
331810,work great . really helpful with restrict exce...,<s> work great really helpful with restrict ex...,"{'work', 'motion', 'excess', 'arm', 'restrict'...",7,9,36,11,"arm excess,swing entire","motion excess,swing entire",work great arm


In [7]:
p_train_df = analay(deepcopy(train_df), 'train')
p_train_df.to_excel('Analay/train_df.xlsx', encoding='utf8')

p_val_df = analay(deepcopy(val_df), 'test')
p_val_df.to_excel('Analay/val_df.xlsx', encoding='utf8')


train row: 196301 | overlap_pos: 0 (58.33)percent| len_lcs: 7: 100%|██████████| 439076/439076 [10:22:24<00:00, 11.76it/s]  
test row: 275696 | overlap_pos: 2 (60.00)percent| len_lcs: 6: 100%|██████████| 48787/48787 [13:19<00:00, 61.04it/s] 


In [8]:
p_train_df.describe()

,cheat_num,overlap,lemm_review_len,lemm_summary_len,len_lcs,percent_lcs,overlap_pos,reveiw_len,summary_len
count,439076.000000,439076.000000,439076.000000,439076.000000,439076.000000,439076.000000,439076.000000,439076.000000,439076.000000
mean,4.280674,7.360400,136.498080,12.454124,6.831496,54.234287,1.415951,136.498080,12.454124
std,1.409944,2.813378,134.499912,3.647949,3.249795,17.184016,0.926050,134.499912,3.647949
min,3.000000,3.000000,20.000000,5.000000,1.000000,5.555556,-1.000000,20.000000,5.000000
25%,3.000000,5.000000,51.000000,10.000000,4.000000,40.000000,0.000000,51.000000,10.000000
50%,4.000000,7.000000,91.000000,12.000000,6.000000,55.555556,2.000000,91.000000,12.000000
75%,5.000000,9.000000,168.000000,14.000000,8.000000,68.750000,2.000000,168.000000,14.000000
max,23.000000,44.000000,1000.000000,61.000000,49.000000,92.307692,2.000000,1000.000000,61.000000


In [12]:
print('reveiw_len < 50 :',len(p_train_df[p_train_df['reveiw_len']<50]))
print('reveiw_len > 1000 :',len(p_train_df[p_train_df['reveiw_len']>1000])) # # max pos, max_enc_steps 

p_train_df['reveiw_len'].describe()

reveiw_len < 50 : 103565
reveiw_len > 1000 : 0


count    439076.000000
mean        136.498080
std         134.499912
min          20.000000
25%          51.000000
50%          91.000000
75%         168.000000
max        1000.000000
Name: reveiw_len, dtype: float64

In [30]:
print('summary_len > 40 :',len(p_train_df[p_train_df['summary_len']>40]))  # max_dec_steps
print('summary_len <= 5 :',len(p_train_df[p_train_df['summary_len']<=5])) # min_dec_steps

p_train_df['summary_len'].describe()

summary_len > 40 : 12
summary_len <= 5 : 20


count    439076.000000
mean         12.454124
std           3.647949
min           5.000000
25%          10.000000
50%          12.000000
75%          14.000000
max          61.000000
Name: summary_len, dtype: float64

In [11]:
print('percent_lcs x < 20 :',
      len(p_train_df[p_train_df['percent_lcs']<20]))  # overlap percent with summary
print('percent_lcs 20 < x < 40 :',
      len(p_train_df[(p_train_df['percent_lcs']>20) & (p_train_df['percent_lcs']<40)]))  # max_dec_steps
print('percent_lcs 40 < x < 60 :',
      len(p_train_df[(p_train_df['percent_lcs']>40) & (p_train_df['percent_lcs']<60)]))  # max_dec_steps
print('percent_lcs 60 < x < 80 :',
      len(p_train_df[(p_train_df['percent_lcs']>60) & (p_train_df['percent_lcs']<80)]))  # max_dec_steps
print('percent_lcs x > 80  :',
      len(p_train_df[(p_train_df['percent_lcs']>80)]))  # max_dec_steps

p_train_df['percent_lcs'].describe()

percent_lcs x < 20 : 4476
percent_lcs 20 < x < 40 : 92028
percent_lcs 40 < x < 60 : 133748
percent_lcs 60 < x < 80 : 146967
percent_lcs x > 80  : 18841


count    439076.000000
mean         54.234287
std          17.184016
min           5.555556
25%          40.000000
50%          55.555556
75%          68.750000
max          92.307692
Name: percent_lcs, dtype: float64

In [34]:
print('percent_lcs x > 50  :',
      len(p_train_df[(p_train_df['percent_lcs']>50)])/len(p_train_df)*100,'%')  # max_dec_steps
print('percent_lcs x < 50  :',
      len(p_train_df[(p_train_df['percent_lcs']<50)])/len(p_train_df)*100,'%')  # max_dec_steps

percent_lcs x > 50  : 54.85405715639205 %
percent_lcs x < 50  : 37.9310187757928 %


In [25]:
'''
overlap => lcs > (20% summary len)
overlap_pos : 0 Top
overlap_pos : 1 Final
overlap_pos : 2 Other Pos
overlap_pos : -1 No overlap
'''

print('overlap Top :',len(p_train_df[p_train_df['overlap_pos'] == 0])/len(p_train_df)*100,'%') 
print('overlap Final :',len(p_train_df[p_train_df['overlap_pos'] == 1])/len(p_train_df)*100,'%') 
print('overlap Other Pos :',len(p_train_df[p_train_df['overlap_pos'] == 2])/len(p_train_df)*100,'%') 
print('No overlap :',len(p_train_df[p_train_df['overlap_pos'] == -1])/len(p_train_df)*100,'%') 

overlap Top : 27.67334128943509 %
overlap Final : 0.0 %
overlap Other Pos : 71.3072452149514 %
No overlap : 1.0194134956135157 %
